Copyright 2023 Google LLC.

SPDX-License-Identifier: Apache-2.0


# License

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# **OneRoster Integration Conformance Tests**


Self link: [Run in Colab](https://colab.research.google.com/github/googleworkspace/oneroster-integration-conformance-tests/blob/main/oneroster_1_1_test_suite.ipynb)![colab](https://cloud.google.com/ml-engine/images/colab-logo-32px.png)

# **Before you begin**

While you are familiarizing yourself with the tests, it might be helpful to run each cell individually. However, when you are ready to submit results to Google, it is preferable to run all of the tests at once for a fully generated test report. Both Colab and Jupyter notebook allow you to run all cells.

The report generated will include a report_card table that will hold the results of each test and a latency_report table that will hold the latency data for each API call. These are the tables that should be submitted to classroom-sis-external@google.com after all the tests are run.

## Setup

Before you run the tests, you will need to include the following in the “Enter credentials” code block:

* Token url to retrieve OAuth 2 credentials
* One Roster URL ending in "/ims/oneroster/v1p1"
* Client ID
* Client secret

You will need an existing teacher email to test the "GetAllTeachers with email filter" code block:

* A teacher email address

## Credentials

In [ ]:
token_url = input("Enter token URL: ")
one_roster_url = input(
    "Enter OneRoster URL (ending in /ims/oneroster/v1p1): ")
client_id = input("Enter your client id: ")
client_secret = input("Enter your client secret: ")
teacher_email_address = input("Enter a teacher email address: ")

In [ ]:
from datetime import datetime, timedelta, timezone
import json
import uuid
import requests
from tabulate import tabulate
from dataclasses import dataclass

report_card = {}
latency_report = {}
test_headers = ["Test", "Result", "Details"]
test_success = "pass"
test_fail = "FAIL"

@dataclass
class Config:
    token_url: str
    one_roster_url: str
    client_id: str
    client_secret: str
    teacher_email_address: str
    class_sourced_id: str = ""
    teacher_sourced_id: str = ""
    line_item_sourced_id: str = ""
    result_sourced_id: str = ""
    term_sourced_id: str = ""

config = Config(
    token_url = token_url,
    one_roster_url = one_roster_url,
    client_id = client_id,
    client_secret = client_secret,
    teacher_email_address = teacher_email_address,
)

@dataclass
class TestResult:
  name: str
  status: str
  details: str = ""

  def data(self):
    return (self.name, self.status, self.details)

def check_status(response, expected_statuses: set[int]):
    codes = "/".join([str(s) for s in expected_statuses])
    test_name = f"Status code {codes}"
    if response.status_code not in expected_statuses:
        print(json.dumps(response.json(), indent=2))
        return TestResult(test_name, status=test_fail, details=f"{response.status_code} returned").data()
    details = f"{response.status_code} returned" if len(expected_statuses) > 1 else ""
    return TestResult(name=test_name, status=test_success, details=details).data()

def print_report(report_card, latency_report):
    table_data = []
    for test_name, results in report_card.items():
        for result in results:
            table_data.append((test_name, result[0], result[1], result[2] if len(result) > 2 else ""))

    print(tabulate(table_data, headers=["Test Name", "Result", "Outcome", "Details"]))
    print("\n")
    print(tabulate(latency_report.items(), headers=["Test Name", "Latency (ms)"]))


# **Basic Grade Sync** [Required]

For best experience, run all at once using ctrl+F9 or `run all` from runtime menu.

## Authorization

In [ ]:
payload = {
    "grant_type": "client_credentials",
    "scope": "https://purl.imsglobal.org/spec/or/v1p1/scope/roster.readonly "
             "https://purl.imsglobal.org/spec/or/v1p1/scope/gradebook.readonly "
             "https://purl.imsglobal.org/spec/or/v1p1/scope/gradebook.createput "
             "https://purl.imsglobal.org/spec/or/v1p1/scope/gradebook.delete"
}
headers = {"Content-Type": "application/x-www-form-urlencoded"}
auth = requests.auth.HTTPBasicAuth(config.client_id, config.client_secret)

response = requests.request(
    "POST", config.token_url, auth=auth, headers=headers, data=payload
)
latency_report["Get OAuth Token"] = response.elapsed.microseconds / 1000

tests = [
  check_status(response, {200})
]

if response.status_code == 200:
  oauth_token = response.json()["access_token"]
  request_headers = {"Authorization": "Bearer " + oauth_token}
  put_request_headers = {
      "Content-Type": "application/json",
      "Authorization": "Bearer " + oauth_token,
  }

  print(f"OAuth token retrieved: {oauth_token}\n")

print(tabulate(tests, headers=test_headers))

report_card["Get OAuth Token"] = tests

## Teachers

### GetAllTeachers

In [ ]:
url = config.one_roster_url + "/teachers"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetAllTeachers"] = response.elapsed.microseconds / 1000

tests = [
    check_status(response, {200})
]

print(tabulate(tests, headers=test_headers))

report_card["GetAllTeachers"] = tests

### GetAllTeachers with email filter

In [ ]:
url = f"{config.one_roster_url}/teachers?filter=email='{config.teacher_email_address}'&limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetAllTeachers w/ email filter"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = [
  check_status(response, {200})
]

if response.status_code == 200:
  test_result = TestResult(name="Validate teacher", status=test_fail)
  if len(data["users"]) != 1:
    test_result.details = f"{len(data['users'])} teachers returned"
  elif data["users"][0].keys() >= {
      "email",
      "sourcedId",
  }:
    test_result.status = test_success
    teacher_sourced_id = data["users"][0]["sourcedId"]
  else:
    test_result.details = "email/sourcedId fields missing"
  tests.append(test_result.data())

print(tabulate(tests, headers=test_headers))

report_card["GetAllTeachers w/ email filter"] = tests

## Classes

### GetClassesForTeacher

In [ ]:
url = f"{config.one_roster_url}/teachers/{teacher_sourced_id}/classes?limit=10000&filter=status='active'"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetClassesForTeacher"] = response.elapsed.microseconds / 1000
data = response.json()
      
tests = [
  check_status(response, {200})
]

if response.status_code == 200:
  config.class_sourced_id = data["classes"][0]["sourcedId"]

print(tabulate(tests, headers=test_headers))

report_card["GetClassesForTeacher"] = tests

## Students

### GetStudentsForClass

In [ ]:
url = f"{config.one_roster_url}/classes/{config.class_sourced_id}/students?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetStudentsForClass"] = response.elapsed.microseconds / 1000
data = response.json()

tests = [
  check_status(response, {200})
]

if response.status_code == 200:
  test_result = TestResult(name="Validate student", status=test_fail)
  if len(data["users"]) == 0:
    test_result.details = "No students found"
  elif data["users"][0].keys() >= {"email", "sourcedId"}:
    student_sourced_id = data["users"][0]["sourcedId"]
    test_result.status = test_success
  else:
    test_result.details = "email/sourcedId fields missing"
  tests.append(test_result.data())


print(tabulate(tests, headers=test_headers))

report_card["GetStudentsForClass"] = tests

## LineItem

### Create

In [ ]:
line_item_sourced_id = str(uuid.uuid4())
now = datetime.now(timezone.utc)
assign_date = now.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
due_date = (now + timedelta(minutes=1)).strftime("%Y-%m-%dT%H:%M:%S.%f")[
    :-3
] + "Z"

url = f"{config.one_roster_url}/lineItems/{line_item_sourced_id}"

payload = json.dumps(
    {
        "lineItem": {
            "sourcedId": line_item_sourced_id,
            "title": "New test item",
            "description": "Test Line Item",
            "resultValueMin": 0,
            "resultValueMax": 100,
            "assignDate": assign_date,
            "dueDate": due_date,
            "class": {"sourcedId": config.class_sourced_id},
            "category": {
                "sourcedId": "84b742b5-739c-31d4-80f2-5613a61509cb-23"
            }
        }
    }
)

response = requests.request(
    "PUT", url, headers=put_request_headers, data=payload
)
latency_report["PutLineItem Create"] = response.elapsed.microseconds / 1000

tests = [
    check_status(response, {201})
]
if response.status_code == 201:
    print(f"Created line item with sourced ID {line_item_sourced_id}\n")

print(tabulate(tests, headers=test_headers))

report_card["PutLineItem Create"] = tests

### Get

In [ ]:
url = f"{config.one_roster_url}/classes/{config.class_sourced_id}/lineItems?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetLineItemsForClass"] = response.elapsed.microseconds / 1000
data = response.json()

tests = [check_status(response, {200})]

if response.status_code == 200:
  if any(sd["sourcedId"] == line_item_sourced_id for sd in data["lineItems"]):
    tests.append(("Get created line item", test_success))
  else:
    tests.append(("Get created line item", test_fail))


print(tabulate(tests, headers=test_headers))

report_card["GetLineItemsForClass"] = tests

### Edit

In [ ]:
url = f"{config.one_roster_url}/lineItems/{line_item_sourced_id}"

payload = json.dumps(
    {
        "lineItem": {
            "sourcedId": line_item_sourced_id,
            "title": (
                "[Amended]"
                " Reallllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllly"
                " Long Google Test Line Item Title"
            ),
            "description": "Test Line Item",
            "resultValueMin": 0,
            "resultValueMax": 100,
            "assignDate": assign_date,
            "dueDate": due_date,
            "class": {"sourcedId": config.class_sourced_id},
        }
    }
)

response = requests.request(
    "PUT", url, headers=put_request_headers, data=payload
)
latency_report["PutLineItem Edit"] = response.elapsed.microseconds / 1000

tests = [check_status(response, {200, 201})]

print(tabulate(tests, headers=test_headers))

if response.status_code == 201:
  print("\nINFO: Modifying existing resource should return status code 200")

report_card["PutLineItem Edit"] = tests

### Get (Assignment Title Limit)

In [ ]:
url = f"{config.one_roster_url}/classes/{config.class_sourced_id}/lineItems?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetLineItemsForClass: Assignment Title Limit"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = [check_status(response, {200})]
if response.status_code == 200:
  for d in data["lineItems"]:
    if d["sourcedId"] == line_item_sourced_id:
      if d:
        tests.append(("Line item title limit", test_success, len(d["title"])))
      else:
        tests.append(("Line item exists", test_fail))
        print(json.dumps(d, indent=2))
      break

print(tabulate(tests, headers=test_headers))

report_card["Assignment Title Limit"] = tests

## Result

### Create

In [ ]:
result_sourced_id = str(uuid.uuid4())

url = f"{config.one_roster_url}/results/{result_sourced_id}"

payload = json.dumps(
    {
        "result": {
            "sourcedId": result_sourced_id,
            "score": 80,
            "comment": "",
            "scoreStatus": "fully graded",
            "scoreDate": assign_date,
            "lineItem": {"sourcedId": line_item_sourced_id},
            "student": {"sourcedId": student_sourced_id},
        }
    }
)

response = requests.request(
    "PUT", url, headers=put_request_headers, data=payload
)
latency_report["PutResult Create"] = response.elapsed.microseconds / 1000

tests = [check_status(response, {200, 201})]
if response.status_code in {200, 201}:
  print(f"Created result with sourced ID {result_sourced_id}\n")

print(tabulate(tests, headers=test_headers))

report_card["PutResult Create"] = tests

### Get

In [ ]:
url = f"{config.one_roster_url}/classes/{config.class_sourced_id}/lineItems/{line_item_sourced_id}/results?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetResultsForLineItem"] = response.elapsed.microseconds / 1000
data = response.json()

tests = [check_status(response, {200})]
if response.status_code == 200:
  if any(sd["sourcedId"] == result_sourced_id for sd in data["results"]):
    tests.append(("Get created result", test_success))
  else:
    tests.append(("Get created result", test_fail))

print(tabulate(tests, headers=test_headers))

report_card["GetResultsForLineItem"] = tests

### Edit

In [ ]:
url = f"{config.one_roster_url}/results/{result_sourced_id}"

payload = json.dumps(
    {
        "result": {
            "sourcedId": result_sourced_id,
            "score": 300,
            "comment": "",
            "scoreStatus": "fully graded",
            "scoreDate": assign_date,
            "lineItem": {"sourcedId": line_item_sourced_id},
            "student": {"sourcedId": student_sourced_id},
        }
    }
)

response = requests.request(
    "PUT", url, headers=put_request_headers, data=payload
)
latency_report["PutResult Edit"] = response.elapsed.microseconds / 1000

tests = [check_status(response, {200, 201})]

print(tabulate(tests, headers=test_headers))

if response.status_code == 201:
  print("\nINFO: Modifying existing resource should return status code 200")

report_card["PutResult Edit"] = tests

### Get (Extra Credit)

In [ ]:
url = f"{config.one_roster_url}/classes/{config.class_sourced_id}/lineItems/{line_item_sourced_id}/results?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetResultsForLineItem Extra Credit"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = [check_status(response, {200})]

for r in data["results"]:
  if r["sourcedId"] == result_sourced_id:
    if r["score"] == 300:
      tests.append(("Result extra credit OK", test_success))
    else:
      tests.append(("Result extra credit OK", test_fail))
      print(json.dumps(r, indent=2))
    break


print(tabulate(tests, headers=test_headers))

report_card["Result Extra Credit"] = tests

### Create (Exempt state)

In [ ]:
result_sourced_id = str(uuid.uuid4())

url = f"{config.one_roster_url}/results/{result_sourced_id}"

payload = json.dumps(
    {
        "result": {
            "sourcedId": result_sourced_id,
            "score": 80,
            "comment": "",
            "scoreStatus": "exempt",
            "scoreDate": assign_date,
            "lineItem": {"sourcedId": line_item_sourced_id},
            "student": {"sourcedId": student_sourced_id},
        }
    }
)

response = requests.request(
    "PUT", url, headers=put_request_headers, data=payload
)
latency_report["PutResult Create Exempt"] = response.elapsed.microseconds / 1000

tests = [check_status(response, {200, 201})]
if response.status_code in {200, 201}:
  print(f"Created result with sourced ID {result_sourced_id}\n")

print(tabulate(tests, headers=test_headers))

report_card["PutResult Create Exempt"] = tests

### Get (Exempt)

In [ ]:
url = f"{config.one_roster_url}/classes/{config.class_sourced_id}/lineItems/{line_item_sourced_id}/results?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetResultsForLineItem Exempt"] = response.elapsed.microseconds / 1000
data = response.json()
print(json.dumps(data, indent=2))
tests = [check_status(response, {200})]
if response.status_code == 200:
  result = next(filter(lambda sd: sd["sourcedId"] == result_sourced_id, data["results"]), None)
  if result and result["scoreStatus"] == "exempt":
    tests.append(("Returned excused state", test_success))
  else:
    tests.append(("Returned excused state", test_fail))

print(tabulate(tests, headers=test_headers))

report_card["GetResultsForLineItem Exempt"] = tests

### Delete

In [ ]:
url = f"{config.one_roster_url}/results/{result_sourced_id}"

response = requests.request("DELETE", url, headers=request_headers)

tests = [check_status(response, {200, 204})]
if response.status_code == 200 or response.status_code == 204:
  latency_report["DeleteResult"] = response.elapsed.microseconds / 1000


print(tabulate(tests, headers=test_headers))

report_card["DeleteResult"] = tests

### Get (Deleted)

In [ ]:
url = f"{config.one_roster_url}/classes/{config.class_sourced_id}/lineItems/{line_item_sourced_id}/results?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetResultsForLineItem (Deleted)"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = []
if not any(sd["sourcedId"] == result_sourced_id for sd in data["results"]):
  tests.append(("Result deleted", test_success))
else:
  tests.append(("Result deleted", test_fail))
  print(json.dumps(data, indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetResultsForLineItem (Deleted)"] = tests

## LineItem (Optional / Cleanup)

### Delete

In [ ]:
url = f"{config.one_roster_url}/lineItems/{line_item_sourced_id}"

response = requests.request("DELETE", url, headers=request_headers)
latency_report["DeleteLineItem"] = response.elapsed.microseconds / 1000

tests = [check_status(response, {200, 204})]

print(tabulate(tests, headers=test_headers))

report_card["DeleteLineItem"] = tests

### Get (Deleted LineItem)

In [ ]:
url = f"{config.one_roster_url}/classes/{config.class_sourced_id}/lineItems?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetLineItemsForClass (Deleted)"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = []
if not any(sd["sourcedId"] == line_item_sourced_id for sd in data["lineItems"]):
  tests.append(("Line item deleted", test_success))
else:
  tests.append(("Line item deleted", test_fail))
  print(json.dumps(data, indent=2))


print(tabulate(tests, headers=test_headers))

report_card["GetLineItemsForClass (Deleted)"] = tests

## Print Report

In [ ]:
print_report(report_card, latency_report)

# **Grading Categories** [Required]

*Only one GET categories endpoint is needed. Please comment out any that are not implemented in order for `run all` to execute successfully.*

## GetAllCategories

In [ ]:
url = f"{config.one_roster_url}/categories?limit=10000"

response = requests.request("GET", url, headers=request_headers, data=payload)
latency_report["GetAllCategories"] = response.elapsed.microseconds / 1000
data = response.json()

tests = [check_status(response, {200})]
if response.status_code == 200:
  category_sourced_id = data["categories"][0]["sourcedId"]


print(tabulate(tests, headers=test_headers))

report_card["GetAllCategories"] = tests

## GetCategoriesForClass

In [ ]:
url = f"{config.one_roster_url}/classes/{config.class_sourced_id}/categories?limit=10000"

response = requests.request("GET", url, headers=request_headers, data=payload)
latency_report["GetCategoriesForClass"] = response.elapsed.microseconds / 1000
data = response.json()

tests = [check_status(response, {200})]
if response.status_code == 200:
  category_sourced_id = data["categories"][0]["sourcedId"]


print(tabulate(tests, headers=test_headers))

report_card["GetCategoriesForClass"] = tests

## PutLineItem w/ category

In [ ]:
line_item_sourced_id = str(uuid.uuid4())

url = f"{config.one_roster_url}/lineItems/{line_item_sourced_id}"

payload = json.dumps(
    {
        "lineItem": {
            "sourcedId": line_item_sourced_id,
            "title": "Google Test Line Item Title",
            "description": "Test Line Item",
            "resultValueMin": 0,
            "resultValueMax": 100,
            "assignDate": assign_date,
            "dueDate": due_date,
            "category": {"sourcedId": category_sourced_id},
            "class": {"sourcedId": config.class_sourced_id},
        }
    }
)

response = requests.request(
    "PUT", url, headers=put_request_headers, data=payload
)
latency_report["PutLineItem w/ Category"] = response.elapsed.microseconds / 1000

tests = [check_status(response, {201})]
if response.status_code == 201:
  print(f"Created line item with sourced ID {line_item_sourced_id}\n")


print(tabulate(tests, headers=test_headers))

report_card["PutLineItem w/ Category"] = tests

## GetLineItemsForClass (Category exists)

In [ ]:
url = f"{config.one_roster_url}/classes/{config.class_sourced_id}/lineItems?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetLineItemsForClass (Category exists)"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = []

for d in data["lineItems"]:
  if d["sourcedId"] == line_item_sourced_id:
    if d and d["category"]["sourcedId"] == category_sourced_id:
      tests.append(("Line item created with category", test_success))
    else:
      tests.append(("Line item created with category", test_fail))
      print(json.dumps(d, indent=2))
    break


print(tabulate(tests, headers=test_headers))

report_card["GetLineItemsForClass (Category exists)"] = tests

## DeleteLineItem (Optional / Cleanup)

In [ ]:
url = f"{config.one_roster_url}/lineItems/{line_item_sourced_id}"

response = requests.request("DELETE", url, headers=request_headers)

tests = [check_status(response, {200, 204})]


print(tabulate(tests, headers=test_headers))

## Print Report

In [ ]:
print_report(report_card, latency_report)

# **Grading Periods** [Required]

## GetAllClasses w/ filter

In [ ]:
url = f"{config.one_roster_url}/classes?filter=sourcedId='{config.class_sourced_id}'&limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetAllClasses w/ filter"] = response.elapsed.microseconds / 1000
data = response.json()

tests = [check_status(response, {200})]
if response.status_code == 200:
  if len(data["classes"]) == 1 and data["classes"][0].keys() >= {
      "terms",
      "sourcedId",
  }:
    term_sourced_id = data["classes"][0]["terms"][0]["sourcedId"]
    tests.append(("Validate class", test_success))
  else:
    tests.append(("Validate class", test_fail))


print(tabulate(tests, headers=test_headers))

report_card["GetAllClasses w/ filter"] = tests

## GetGradingPeriodsForTerm

In [ ]:
url = f"{config.one_roster_url}/terms/{term_sourced_id}/gradingPeriods?limit=10000"

response = requests.request("GET", url, headers=request_headers)
latency_report["GetGradingPeriodsForTerm"] = (
    response.elapsed.microseconds / 1000
)
data = response.json()

tests = [check_status(response, {200})]
if response.status_code == 200:
  if data["academicSessions"][0].keys() >= {
      "sourcedId",
      "title",
      "startDate",
      "endDate",
      "schoolYear",
  }:
    tests.append(("Validate grading period", test_success))
  else:
    tests.append(("Validate grading period", test_fail))

  tests.append(("Academic sessions all grading period types", test_success))
  for d in data["academicSessions"]:
    if d["type"] != "gradingPeriod":
      tests.append(("Academic sessions all grading period types", test_fail))
      print(json.dumps(d, indent=2))
      break


print(tabulate(tests, headers=test_headers))

report_card["GetGradingPeriodsForTerm"] = tests

## Print Report

In [ ]:
print_report(report_card, latency_report)